# Modeling

In [1]:
import ppdet
# ppdet.model_zoo.list_model(['picodet'])
# ppdet.model_zoo.list_model(['ssd'])


In [20]:
model = ppdet.model_zoo.get_model('picodet\picodet_s_320_coco_lcnet', pretrained=False)


In [24]:
model_ssd = ppdet.model_zoo.get_model('ssd\ssd_mobilenet_v1_300_120e_voc', pretrained=False)

In [2]:
model_custom = ppdet.model_zoo.get_model('picodet\picodet_s_300_custom', pretrained=False)

# Below for QAT(ACT)

In [13]:
# export
config_yml = r"configs\ssd\ssdlite_mobilenet_v3_small_custom.yml"
config_yml = r"configs\ssd\ssd_mobilenet_v1_custom.yml"
config_yml = r"configs/picodet/picodet_s_300_custom.yml"
weight_path = r"..\ssdlite_models\before_convert\best_model.pdparams"
weight_path = r"..\ssd_models\before_convert\best_model.pdparams"
weight_path = r"output/picodet_s_300_custom/best_model.pdparams"
!python tools/export_model.py -c $config_yml -o weights=$weight_path

[04/06 18:10:23] ppdet.utils.checkpoint INFO: Finish loading model weights: output/picodet_s_300_custom/best_model.pdparams
[04/06 18:10:23] ppdet.engine INFO: Export inference config file to output_inference\picodet_s_300_custom\infer_cfg.yml
[04/06 18:10:26] ppdet.engine INFO: Export model and saved in output_inference\picodet_s_300_custom


In [ ]:
act_yml = "ssdlite_qat.yml"
!python deploy/auto_compression/run.py --config_path=$act_yml --save_dir='./output/ssd_lite_qat_distilled'  

# Test

In [4]:
import numpy as np
picodet_weight = np.fromfile(r"F:\CICC\env\PaddleDetection\output\picodet_s_288_custom_trained_input-fp32\model_final.pdparams")
ssd_weight = np.fromfile(r"F:\CICC\env\ssd_models\after_convert\ssd_mobilenet_v1_custom\model.pdiparams")

In [11]:
import paddle
paddle.enable_static()
exe = paddle.static.Executor(paddle.CPUPlace())
[inference_program, feed_target_names, fetch_targets] = (paddle.static.load_inference_model(r"F:\CICC\env\picodet_models\after_convert\picodet_s_300_custom_fp-norm\model", exe))

# View Sensitivity Data


In [13]:
import pickle
data = pickle.load(open("sensitivity_0.data", "rb"))